## **门控循环单元（GRU）**

循环神经网络中，当时间步数较大或时间步较小时，循环神经网络的梯度容易出现衰减或爆炸。虽然裁剪梯度可以应对梯度爆炸，但是无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中较验捕捉时间序列中时间步较大的依赖关系。

**门控循环神经网络（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。它通过可以学习的门来控制信息的流动。** 其中，门控循环单元（gated recurrent unit,GRU）是一种常用的门控循环神经网络。


1、重置门有助于捕捉时间序列里短期的依赖关系。

2、更新门有助于捕捉时间序列里长期的依赖关系。


### <font color='blue'>**一、门控循环单元**</font>

引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式。


<font size=4 color='blue'>**1、重置门和更新门**</font>

如图6.4，门控循环单元中的重置门和更新门的输入均为当前时间步输入 $X_t$ 与上一时间步隐藏状态 $H_{t-1}$，输出由激活函数为sigmoid函数的全连接层计算得到。

具体来说，假设隐藏单元个数为 $h$，给定时间步 $t$ 的小批量输入 $X_t\in \mathcal{R}^{n\times d}$（样本数为 $n$，输入个数为 $d$）和上一时间步隐藏状态 $H_{t-1} \in \mathcal{R}^{n\times h}$。重置门 $R_t\in \mathcal{R}^{n\times h}$ 和更新门 $Z_t\in \mathcal{R}^{n\times h}$ 的计算如下：
#### $$\begin{align} & R_t = \sigma(X_tW_{xr} + H_{t-1}W_{hr} + b_r)\\ & Z_t = \sigma(X_tW_{xz} + H_{t-1}W_{hz} + b_z) \end{align}$$
其中，$W_{xr},W_{xz}\in \mathcal{R}^{d\times h}$ 和 $W_{hr},W_{hz} in \mathcal{R}^{h\times h}$ 是权重参数， $b_r,b_z in \mathcal{R}^{1\times h}$ 是偏差参数。由于sigmoid函数可以将元素的值变换到0-1之间，因此，重置门 $R_t$ 和更新门 $Z_t$ 中每个元素的值域都是 $[0,1]$ $$ $$ 

<img src='gru.png' width = 500></img>

<font size=4 color='blue'>**2、候选隐藏状态**</font>

接下来，门控循环单元将计算候选隐藏状态来辅助稍后的隐藏状态计算。如图6.5所示，我们**将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法** （符号为 $\varTheta$）。如果重置门中元素接近0，意味着重置对应隐藏状态元素为0，即丢弃上一时间步的隐藏状态。如果元素接近1，那么表示保留上一时间步的隐藏状态。然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数 tanh的全连接层计算出候选隐藏状态，其所有元素的值域为 $[-1,1]$

具体来说，时间步 $t$的候选隐藏状态 $\tilde{H}_t \in \mathcal{R}^{n\times h}$ 的计算为
#### $$ \tilde{H}_t = tanh(X_tW_{xh} + (R_t \varTheta H_{t-1})W_{hh} + b_h) $$ 
其中，$W_{xh}\in \mathcal{R}^{d\times h}$ 和 $W_{hh}\in \mathcal{R}^{h\times h}$ 是权重参数， $h_h\in \mathcal{R}^{1\times h}$ 是偏差参数。

从上面的公式可以看出，**重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。** 而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。


<img src = 'gru2.png' width = 600></img>

<font size=4 color='blue'>**3、隐藏状态**</font>

时间步 $t$ 的隐藏状态 $H_t\in \mathcal{R}^{n\times h}$ 的计算使用当前时间步的更新门 $Z_t$ 来对上一时间步的隐藏状态 $H_{t-1}$ 和当前时间步的候选隐藏状态 $\tilde{H}_t$ 做组合：
### $$H_t = Z_t \varTheta H_{t-1} + (1-Z_t)\varTheta \tilde{H_t}$$

值得注意的是，更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，如图6.6所示。

假设更新门在时间步 $t'$ 到 $t(t'<t)$ 之间一直近似1。那么，在时间步 $t'$ 到 $t$ 之间的输入信息几乎没有流入时间步 $t$ 的隐藏状态 $H_t$。实际上，这可以看作是较早时刻的隐藏状态 $H_{t'-1}$ 一直通过时间保存并传递至当前时间步 $t$。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

<img src = 'gru3.png' width = 600></img>